In [26]:
import pandas as pd
import numpy as np

data = pd.read_csv('checkins.dat', delimiter="|", header=0)

In [28]:
data.columns = data.columns.str.strip()
data.columns

Index([u'id', u'user_id', u'venue_id', u'latitude', u'longitude',
       u'created_at'],
      dtype='object')

In [35]:
data["latitude"] = data["latitude"].str.strip()
data["latitude"].replace('', np.nan, inplace=True)
data.dropna(inplace=True)

In [32]:
data.dropna(inplace=True)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null object
longitude     396634 non-null object
created_at    396634 non-null object
dtypes: int64(3), object(3)
memory usage: 21.2+ MB


In [39]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [40]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [48]:
data_t = data.head(100000)

In [52]:
coords = data_t[["latitude", "longitude"]]

In [54]:
coords.head()

,latitude,longitude
1,38.8951118,-77.0363658
3,33.800745,-84.41052
7,45.5234515,-122.6762071
9,40.764462,-111.904565
10,33.4483771,-112.0740373


In [55]:
ms = MeanShift(bandwidth=0.1)
ms.fit(coords)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [73]:
len(ms.cluster_centers_)

3229

In [75]:
coords = [ms.cluster_centers_[i] for i in range(0, len(ms.cluster_centers_)) if np.bincount(ms.labels_)[i] > 15]

In [79]:
coords[0:5]

[array([ 40.7177164 , -73.99183542]),
 array([  33.44943805, -112.00213969]),
 array([  33.44638027, -111.90188756]),
 array([ 41.87824378, -87.62984336]),
 array([  37.68868157, -122.40933037])]

In [65]:
len(ms.labels_)

100000

In [77]:
offices = [[33.751277, -118.188740],
            [25.867736, -80.324116],
            [51.503016, -0.075479],
            [52.378894, 4.885084],
            [39.366487, 117.036146],
            [-33.868457, 151.205134]]

In [78]:
offices

[[33.751277, -118.18874],
 [25.867736, -80.324116],
 [51.503016, -0.075479],
 [52.378894, 4.885084],
 [39.366487, 117.036146],
 [-33.868457, 151.205134]]

In [81]:
import scipy as sc
sc.spatial.distance.euclidean(offices[0], offices[1])

38.676607521343655

In [93]:
allmin = []
for coord in coords:
    min_dist = 1000.0
    for office in offices:
        dist = sc.spatial.distance.euclidean(office, coord)
        if dist < min_dist: min_dist = dist
    allmin.append({"lt": coord[0], "lg": coord[1], "dist": min_dist})

In [102]:
pd.DataFrame(allmin).sort_values(['dist'], ascending=[1])

,dist,lg,lt
410,0.007835,151.204776,-33.860630
374,0.009353,4.892317,52.372964
402,0.022674,-80.318891,25.845672
58,0.050058,-0.125537,51.502991
51,0.070848,-118.148924,33.809878
29,0.134109,-80.217938,25.785812
166,0.167406,-80.283429,25.705350
92,0.188876,-80.199991,26.010098
87,0.195779,-118.048928,33.888325
42,0.211811,-118.362091,33.872986
